In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt
from torch import nn, optim
from google.colab import drive
drive.mount("/content/drive")

# Define datasets and models
class FeatureDataset(Dataset):
    def __init__(self, features, labels, paths):
        self.features = features
        self.labels = labels
        self.paths = paths

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx], self.paths[idx]


class SimpleNN(nn.Module):
    def __init__(self, layers, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.layers = nn.ModuleList()
        prev_dim = input_dim
        for layer_dim in layers:
            self.layers.append(nn.Linear(prev_dim, layer_dim))
            self.layers.append(nn.ReLU())
            prev_dim = layer_dim
        self.layers.append(nn.Linear(prev_dim, output_dim))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


# Load datasets
train_features_path = "/content/drive/MyDrive/antispoofing/features-LCC_train-dinov2_vitb14.npy"
train_labels_path = "/content/drive/MyDrive/antispoofing/labels-LCC_train-dinov2_vitb14.npy"
train_paths_path = "/content/drive/MyDrive/antispoofing/paths-LCC_train-dinov2_vitb14.npy"

dev_features_path = "/content/drive/MyDrive/antispoofing/features-LCC_dev-dinov2_vitb14.npy"
dev_labels_path = "/content/drive/MyDrive/antispoofing/labels-LCC_dev-dinov2_vitb14.npy"
dev_paths_path = "/content/drive/MyDrive/antispoofing/paths-LCC_dev-dinov2_vitb14.npy"

train_features = np.load(train_features_path)
train_labels = np.load(train_labels_path)
train_paths = np.load(train_paths_path, allow_pickle=True)

dev_features = np.load(dev_features_path)
dev_labels = np.load(dev_labels_path)
dev_paths = np.load(dev_paths_path, allow_pickle=True)

train_features = torch.tensor(train_features, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)
dev_features = torch.tensor(dev_features, dtype=torch.float32)
dev_labels = torch.tensor(dev_labels, dtype=torch.long)

train_dataset = FeatureDataset(train_features, train_labels, train_paths)
dev_dataset = FeatureDataset(dev_features, dev_labels, dev_paths)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False)

# Model configurations
input_dim = 768
output_dim = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Mounted at /content/drive


In [ ]:

nn_configurations = {
    "2-layer NN": [128, 64],
    "3-layer NN": [256, 128, 64],
    "4-layer NN": [512, 256, 128, 64],
    "5-layer NN": [512, 256, 128, 64, 32],
}

# Training and evaluation
results = {}

for name, layers in nn_configurations.items():
    print(f"Training {name}...")
    model = SimpleNN(layers, input_dim, output_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Training
    epochs = 100
    for epoch in range(epochs):
        model.train()
        for batch_features, batch_labels, _ in train_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    all_labels = []
    all_predictions = []
    misclassified_paths = []
    with torch.no_grad():
        for batch_features, batch_labels, batch_paths in dev_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            outputs = model(batch_features)
            _, predictions = torch.max(outputs, 1)
            all_labels.extend(batch_labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())
            for i, (label, pred, path) in enumerate(zip(batch_labels.cpu().numpy(), predictions.cpu().numpy(), batch_paths)):
                if label != pred:
                    misclassified_paths.append((path, label, pred))

    accuracy = accuracy_score(all_labels, all_predictions) * 100
    results[name] = {"accuracy": accuracy, "misclassified_paths": misclassified_paths}
    print(f"{name} Accuracy: {accuracy:.2f}%")

# Plot accuracy comparison
plt.figure(figsize=(10, 6))
plt.bar(results.keys(), [result["accuracy"] for result in results.values()])
plt.title("Accuracy Comparison Across Models")
plt.xlabel("Model")
plt.ylabel("Accuracy (%)")
plt.grid(axis="y")
plt.show()

# Display confusion matrices
for name, result in results.items():
    print(f"Confusion Matrix for {name}")
    conf_matrix = confusion_matrix(all_labels, all_predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=[0, 1])
    disp.plot(cmap="viridis", values_format="d")
    plt.title(f"Confusion Matrix - {name}")
    plt.show()


In [9]:
class ImprovedNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ImprovedNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)

        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(256, 128)
        self.relu3 = nn.ReLU()

        self.fc4 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.relu3(x)

        x = self.fc4(x)
        return x

name = "3 - Layer Improved Model"
print(f"Training {name}...")
model = ImprovedNN(input_dim, output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

# Training
epochs = 300
for epoch in range(epochs):
    model.train()
    for batch_features, batch_labels, _ in train_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
all_labels = []
all_predictions = []
misclassified_paths = []
with torch.no_grad():
    for batch_features, batch_labels, batch_paths in dev_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        outputs = model(batch_features)
        _, predictions = torch.max(outputs, 1)
        all_labels.extend(batch_labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())
        for i, (label, pred, path) in enumerate(zip(batch_labels.cpu().numpy(), predictions.cpu().numpy(), batch_paths)):
            if label != pred:
                misclassified_paths.append((path, label, pred))

accuracy = accuracy_score(all_labels, all_predictions) * 100
results[name] = {"accuracy": accuracy, "misclassified_paths": misclassified_paths}
print(f"{name} Accuracy: {accuracy:.2f}%")

Training improved model...
5-layer NN Accuracy: 97.35%


In [11]:
len(results.keys())

4

In [6]:
class LightweightCNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LightweightCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        self.fc1 = nn.Linear(input_dim // 4 * 64, 128)
        self.relu_fc1 = nn.ReLU()
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = x.unsqueeze(1)  # Adding channel dimension for Conv1D
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.relu_fc1(x)
        x = self.fc2(x)
        return x

name = "Lightweight CNN model"
print(f"Training {name}...")
model = LightweightCNN(input_dim, output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

# Training
epochs = 300
for epoch in range(epochs):
    model.train()
    for batch_features, batch_labels, _ in train_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
all_labels = []
all_predictions = []
misclassified_paths = []
with torch.no_grad():
    for batch_features, batch_labels, batch_paths in dev_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        outputs = model(batch_features)
        _, predictions = torch.max(outputs, 1)
        all_labels.extend(batch_labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())
        for i, (label, pred, path) in enumerate(zip(batch_labels.cpu().numpy(), predictions.cpu().numpy(), batch_paths)):
            if label != pred:
                misclassified_paths.append((path, label, pred))

accuracy = accuracy_score(all_labels, all_predictions) * 100
results[name] = {"accuracy": accuracy, "misclassified_paths": misclassified_paths}
print(f"{name} Accuracy: {accuracy:.2f}%")

Training improved model...
Light Weight model Accuracy: 94.81%


Light Weight model Accuracy: 42.10%
